# Comment expliquer la violence par armes à feu aux Etats-Unis ?

*Python pour la data science - Matis Bruneau, Eloïse Leroux, Matthieu Grenier - 2023-2024*

Au cours de ce projet, nous essayons de fournir des éléments de réponse autour de la question de la violence par armes à feu aux Etats-Unis. Autrement dit, est-il possible d'identifier certains facteurs à l'origine de ces indicidents ? Deux questions majeures traversent cette analyse : 1) Peut-on prédire la fréquence de ces incidents ? 2) Peut-on prédire les conséquences humaines et matérielles de ces incidents ?

Ce projet est motivé par une [base de données](https://github.com/jamesqo/gun-violence-data) recensant tous les incidents impliquant des armes à feu ayant survenu aux Etats-Unis entre 2013 et 2018, soit près de 260 000 entrées.


Afin d'en proposer une étude cohérente, nous nous appuyons sur plusieurs autres sources de données : une série de données socio-économiques auprès des comtés américains, leur positionnement politique, le nombre de *abckground checks* par Etat, et une synthèse des différences de législation entre Etats.

## 1 - Récupération et traitement des données

## 2 - Prédire la violence par armes à feu à partir des caractéristiques des individus

### A) Sur la fréquence des incidents

### B) Sur les conséquences des incidents


## 3 - Prédire la violence par armes à feu à partir des caractéristiques structurelles des comtés

### A) Sur la fréquence des incidents

### B) Sur les conséquences des incidents


## 4 - Tentative de modélisation